In [16]:
import cv2
import numpy as np
import os

In [3]:
def are_pixels_similar(pixel1, pixel2, threshold=10):
    distance = np.linalg.norm(pixel1 - pixel2)
    return distance < threshold

def remove_lines(chess_board):
    while not are_pixels_similar(chess_board[10][0], chess_board[10][1]):  # left 
        chess_board = chess_board[:, 1:]
    while not are_pixels_similar(chess_board[0][10], chess_board[1][10]):  # top
        chess_board = chess_board[1:, :]
    while not are_pixels_similar(chess_board[-11][-1], chess_board[-11][-2]):  # right
        chess_board = chess_board[:, :-1]
    while not are_pixels_similar(chess_board[-1][-11], chess_board[-2][-11]):  # bottom
        chess_board = chess_board[:-1, :]
    return chess_board

In [10]:
def preprocess_cells(filename, piece_setup, player_side):
    screenshot = cv2.imread(filename)
    screenshot_gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY)
    
    sigma=0.33
    v = np.mean(screenshot_gray)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    
    edges = cv2.Canny(screenshot_gray, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
    
    (x, y, w, h) = cv2.boundingRect(contours[0])   
    best_area = cv2.contourArea(contours[0])
    for i, cnt in enumerate(contours):
        epsilon = 0.01 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if len(approx) == 4:
            area = cv2.contourArea(cnt)
            area_diff = abs(area - best_area) / best_area
            if area_diff > 0.05:
                break
            (x, y, w, h) = cv2.boundingRect(cnt)

    chess_board = screenshot[y:y+h, x:x+w]
    chess_board = remove_lines(chess_board)
    chess_board = cv2.resize(chess_board, (1024, 1024))
    
    columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    rows = ['1', '2', '3', '4', '5', '6', '7', '8']
    step = (chess_board.shape[0]//8)
    
    for row in range(len(rows)):
        for col in range(len(columns)):
            cell_image = chess_board[row*step:(row+1)*step, col*step:(col+1)*step]
            if player_side == "white":
                cell_label = f"{columns[col]}{rows[7-row]}"
            else:
                cell_label = f"{columns[7-col]}{rows[row]}"
                
            if cell_label in piece_setup.keys():
                filename = f"../data/cells/pieces/{piece_setup[cell_label]}.png"
                print(filename)
                cv2.imwrite(filename, cell_image)

    print("Cells processed successfully")
            

In [14]:
# initial_piece_setup = {
#     'a1': 'lichess_white_rook_on_black',   'b1': 'lichess_white_knight_on_white', 'c1': 'lichess_white_bishop_on_black', 'd1': 'lichess_white_queen_on_white',
#     'e1': 'lichess_white_king_on_black',   'f1': 'lichess_white_bishop_on_white', 'g1': 'lichess_white_knight_on_black', 'h1': 'lichess_white_rook_on_white',
#     'a2': 'lichess_white_pawn_on_white',   'b2': 'lichess_white_pawn_on_black',
# 
#     'a8': 'lichess_black_rook_on_white',   'b8': 'lichess_black_knight_on_black', 'c8': 'lichess_black_bishop_on_white', 'd8': 'lichess_black_queen_on_black',
#     'e8': 'lichess_black_king_on_white',   'f8': 'lichess_black_bishop_on_black', 'g8': 'lichess_black_knight_on_white', 'h8': 'lichess_black_rook_on_black',
#     'a7': 'lichess_black_pawn_on_black',   'b7': 'lichess_black_pawn_on_white'
# }

piece_setup = {
    'e1': "lichess_white_queen_on_black",
    'e2': "lichess_white_king_on_white",
    # 'e2': "lichess_white_king_on_white",
    # 'e7': "lichess_black_king_on_black",
}

preprocess_cells(
    filename="../data/cells/screenshots/lichess_white_queen_king_another.png",
    player_side="white",
    piece_setup=piece_setup
)


../data/cells/pieces/lichess_white_king_on_white.png
../data/cells/pieces/lichess_white_queen_on_black.png
Cells processed successfully


In [37]:
def preprocess_bottom_cell(filename, file):
    screenshot = cv2.imread(filename)
    screenshot_gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY)
    
    sigma=0.33
    v = np.mean(screenshot_gray)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    
    edges = cv2.Canny(screenshot_gray, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:5]
    
    (x, y, w, h) = cv2.boundingRect(contours[0])   
    best_area = cv2.contourArea(contours[0])
    for i, cnt in enumerate(contours):
        epsilon = 0.01 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        if len(approx) == 4:
            area = cv2.contourArea(cnt)
            area_diff = abs(area - best_area) / best_area
            if area_diff > 0.05:
                break
            (x, y, w, h) = cv2.boundingRect(cnt)

    chess_board = screenshot[y:y+h, x:x+w]
    chess_board = remove_lines(chess_board)
    chess_board = cv2.resize(chess_board, (1024, 1024))
    
    step = (chess_board.shape[0]//8)
    row, col = 7, 3
    bottom_cell = chess_board[row*step : (row+1)*step, col*step : (col+1)*step]
    
    new_filename = f"../data/cells/player_side_cell/{file}"
    cv2.imwrite(new_filename, bottom_cell)
            

In [40]:
for dir in os.walk("../data/sites"):
    for file in dir[2]:
        file_path = os.path.join(dir[0], file)
        if "chesscom" in file_path:
            player_side = "black"
        preprocess_bottom_cell(file_path, file)
